## Imports

In [1]:
import os

In [2]:
from fuzzywuzzy import fuzz

## Config

In [3]:
feature_list_id = 'fuzzy'

In [4]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep

## Read Data

In [5]:
df_questions_train = pd.read_csv(data_folder + 'train.csv').fillna('')
df_questions_test = pd.read_csv(data_folder + 'test.csv').fillna('')

In [6]:
question_tokens_train = load_json(preproc_data_folder + 'question_tokens_train.json')
question_tokens_test = load_json(preproc_data_folder + 'question_tokens_test.json')

## Build Features

In [7]:
A = np.ones((10, 10))

In [8]:
def build_features(df_questions_original, questions_tokenized):
    num_pairs = len(questions_tokenized)
    num_features = 5
    
    X = np.zeros((num_pairs, num_features), dtype=float)
    
    for index, pair in progressbar(enumerate(questions_tokenized), size=num_pairs):
        # Fuzzy ratio.
        X[index, 0] = fuzz.ratio(pair['question1'], pair['question2'])

        # Fuzzy partial ratio.
        X[index, 1] = fuzz.partial_ratio(pair['question1'], pair['question2'])
        
        # Fuzzy token sort ratio.
        X[index, 2] = fuzz.token_sort_ratio(pair['question1'], pair['question2'])
    
        # Fuzzy token set ratio.
        X[index, 3] = fuzz.token_set_ratio(pair['question1'], pair['question2'])
        
        # Fuzzy partial token sort ratio.
        X[index, 4] = fuzz.partial_token_sort_ratio(pair['question1'], pair['question2'])
            
    # Normalize the ratios to [0 - 1] range.
    return X / 100

In [9]:
X_train = build_features(df_questions_train, question_tokens_train)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [10]:
save(X_train, features_data_folder + f'X_train_{feature_list_id}.pickle')

In [11]:
X_test = build_features(df_questions_test, question_tokens_test)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [12]:
save(X_test, features_data_folder + f'X_test_{feature_list_id}.pickle')